<a href="https://colab.research.google.com/github/chenlizzhou/Supply-Chain-/blob/main/Tailored_Sourcing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import pandas as pd
import numpy as np

In [2]:
demand= [1000, 300, 100, 50]
trans_cost= [100, 100, 100, 100]
order_cost= [10, 20, 25, 25]
unit_cost= [50, 60, 30, 30]
hold_cost_percent= [0.2, 0.2, 0.2, 0.2]

### Question 1 Products are sourced independently


In [3]:
# Use the formula Q=sqrt(2DS/H)
# Calculate holding cost per unit
hold_cost = [uc * hc for uc, hc in zip(unit_cost, hold_cost_percent)]
per_order_cost= [(t+ o) for t, o in zip(trans_cost, order_cost)]

# Calculate optimal order size Q for each product
optimal_Q1 = [round(math.sqrt((2*d*s)/h), 0) for d, s, h in zip(demand, per_order_cost, hold_cost)]

for i, q in enumerate(optimal_Q1, start=1):
    print(f'For product {i}, the optimal order size is {q:.1f}')

For product 1, the optimal order size is 148.0
For product 2, the optimal order size is 77.0
For product 3, the optimal order size is 65.0
For product 4, the optimal order size is 46.0


In [4]:
order_freq= [round((d/q),4) for d, q in zip(demand, optimal_Q1)]
order_freq

[6.7568, 3.8961, 1.5385, 1.087]

In [5]:
# annual operational cost = (holding + ordering)
# q/2 = average inventory
annual_h_cost = [(q / 2) * H for q, H in zip(optimal_Q1, hold_cost)]
annual_o_cost = [round(of * s, 2) for of, s in zip(order_freq, per_order_cost)]

# Calculate total annual operational cost for each product
total_annual_op_cost = [round(hcost + ocost, 2) for hcost, ocost in zip(annual_h_cost, annual_o_cost)]

# Preparing formatted output for each product
for i, (h_cost, o_cost, total_cost) in enumerate(zip(annual_h_cost, annual_o_cost, total_annual_op_cost), start=1):
    print(f'For product {i}, the annual holding cost is {h_cost:.2f}, the annual ordering cost is {o_cost:.2f}, and the total annual operational cost is {total_cost:.2f}')

sum_total_1= sum(total_annual_op_cost)
print(f'Total annual operational cost is', sum_total_1)

For product 1, the annual holding cost is 740.00, the annual ordering cost is 743.25, and the total annual operational cost is 1483.25
For product 2, the annual holding cost is 462.00, the annual ordering cost is 467.53, and the total annual operational cost is 929.53
For product 3, the annual holding cost is 195.00, the annual ordering cost is 192.31, and the total annual operational cost is 387.31
For product 4, the annual holding cost is 138.00, the annual ordering cost is 135.88, and the total annual operational cost is 273.88
Total annual operational cost is 3073.97


### Question 2 All four products are sourced with the same frequency

In [6]:
# If all products are sourced with the same frequency, we will need to calculate the cost for ordering all products one time
trans_cost = 100

# Calculate total order cost when ordering together
order_together_cost = sum(order_cost) + trans_cost
S= order_together_cost
print(f'The cost of ordering all products at one time is ${S}')

The cost of ordering all products at one time is $180


In [7]:
DhC = sum(d* H for d, H in zip(demand, hold_cost))

# Calculate optimal order frequency n
n = math.sqrt((DhC) / (2 * S))
n

6.346477588219924

In [8]:
optimal_Q2= [round(d/n, 0) for d in demand]
for i, Q2 in enumerate(optimal_Q2, start=1):
  print(f'For product{i}, the optimal order size is {Q2}')

For product1, the optimal order size is 158.0
For product2, the optimal order size is 47.0
For product3, the optimal order size is 16.0
For product4, the optimal order size is 8.0


In [9]:
# annual operational cost = (holding + ordering)
# q/2 = average inventory
annual_h_cost2 = [(q / 2) * H for q, H in zip(optimal_Q2, hold_cost)]
annual_o_cost2 = n*S

# Calculate total annual operational cost for each product
sum_total_2 = sum(annual_h_cost2)+ annual_o_cost2


print(f'Total annual operational cost is', round(sum_total_2, 2))

Total annual operational cost is 2286.37


### Question 3 Order Frequencies are determined by the tailored aggregation stragegy

In [10]:
# Based on the result of question 1, product 1 is the most freq ordered product as 6.758 times per year
# recompute the frequencies for product 2,3,4 using only product cost
fre1_4= 6.7568
demand2_4= demand[1:]
order_cost2_4= order_cost[1:]
hold_cost2_4= hold_cost[1:]

Q2_Q4 = [round(math.sqrt((2*d*s)/h), 0) for d, s, h in zip(demand2_4, order_cost2_4, hold_cost2_4)] # only contains the result from Product 2 to Product 4
fre2_4= [d/q for d, q in zip(demand2_4, Q2_Q4)]

fre2_4

[9.375, 3.4482758620689653, 2.5]

In [11]:
# Calculate the m for each product
all_m_number= [1]
m_number2_4= []
for i, f in enumerate(fre2_4, start=2):
    m = math.ceil(fre1_4/ f)
    all_m_number.append(m)
    m_number2_4.append(m)

print(all_m_number)

[1, 1, 2, 3]


In [12]:
# Calculate the n  for each product
order_cost1= 110
DD= sum(d * H * m for d, H, m in zip(demand, hold_cost, all_m_number))
SS= 2*(order_cost1+ sum(s/m for s, m in zip(order_cost2_4, m_number2_4)))
n1= round(math.sqrt(DD/SS), 1)
n1

7.2

In [13]:
n2_4= [n1/ m for m in m_number2_4]
all_frequency= [n1]
all_frequency.extend(n2_4)
all_frequency

[7.2, 7.2, 3.6, 2.4]

In [14]:
optimal_Q3= [round(d/f, 0) for d, f in zip(demand, all_frequency)]
optimal_Q3

[139.0, 42.0, 28.0, 21.0]

In [15]:
S_cost= [order_cost1]
S_cost.extend(order_cost2_4)

annual_o_cost3 = sum(S*f for S, f in zip(S_cost, all_frequency))
annual_h_cost3= sum(q/2*H for q, H in zip(optimal_Q3, hold_cost))
annual_total_3= annual_o_cost3+ annual_h_cost3

print(f'Total annual operational cost is {annual_total_3}')

Total annual operational cost is 2180.0
